In [1]:
!pip install chromadb


In [2]:
import chromadb
from transformers import AutoTokenizer, AutoModel
import torch
import random
from transformers import AutoModelForCausalLM

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Code

(Codet5 , graph code bert , qween)

In [ ]:
model_codet5_n = "Salesforce/codet5-base"
tokenizer_codet5 = AutoTokenizer.from_pretrained(model_codet5_n)
model_codet5 = AutoModel.from_pretrained(model_codet5_n)

#Code t5 code (Sameh Notebook)
def get_embeddings_CodeT5_Code(code,model,tokenizer):
    inputs = tokenizer(code, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        encoder_outputs = model.encoder(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"]
        )
    return encoder_outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [41]:
tokenizer_graphBert = AutoTokenizer.from_pretrained('microsoft/graphcodebert-base')
model_graphBert = AutoModel.from_pretrained('microsoft/graphcodebert-base')

Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
#Graph Bert code (Jannah Notebook)
def get_embeddings_GraphBert_Code(code_snippet,model,tokenizer):

    inputs = tokenizer(code_snippet, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

In [4]:
#Qween code (Amjad Notebook)
model_qween_n = "Qwen/Qwen2.5-Coder-7B-Instruct"
model_qween = AutoModelForCausalLM.from_pretrained(
    model_qween_n,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer_qween = AutoTokenizer.from_pretrained(model_qween_n)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [5]:
def get_embeddings_Qween_Codee(texts , model ,tokenizer):

    prompt = texts
    messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant in C++."},
        {"role": "user", "content": prompt}
    ]

    # Convert messages to formatted text
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize input
    inputs = tokenizer([text], return_tensors="pt")
    print(model)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    # Mean pooling to get embeddings
    embeddings = outputs.hidden_states[-1].float().mean(dim=1).cpu().numpy()

    return embeddings


In [66]:
def Get_similar_Code(code,model_name , model, tokenizer, db_path, collection_name):
    try:
        client = chromadb.PersistentClient(path=db_path)
        collection = client.get_collection(name=collection_name)

        if model_name.lower() == "graphcodebert":
            embedding = get_embeddings_GraphBert_Code(code, model,tokenizer)
        elif model_name.lower() == "codet5":
            embedding = get_embeddings_CodeT5_Code(code, model,tokenizer)
        elif model_name.lower() == "qween":
            embedding = get_embeddings_Qween_Codee(code, model,tokenizer)
        else:
            raise ValueError(f"Unsupported model name: {model_name}")

        embedding_list = embedding.flatten().tolist()

        total_documents = collection.count()

        results = collection.query(
            query_embeddings=[embedding_list],
            n_results=total_documents
        )

        # Fetch distances and normalize them
        distances = results["distances"][0]
        min_distance = min(distances)
        max_distance = max(distances)

        def normalize_distance(distance):
            # Normalize distances to the range [0, 1]
            return 1 - ((distance - min_distance) / (max_distance - min_distance))

        similar_code_snippets = [
            {
                "id": item_id,
                "code": document,
                "similarity": normalize_distance(distance)
            }
            for item_id, document, distance in zip(
                results["ids"][0],
                results["documents"][0],
                results["distances"][0]
            )
        ]

        # Sort by cosine similarity (descending order)
        similar_code_snippets.sort(key=lambda x: x["similarity"], reverse=True)

        for snippet in similar_code_snippets:
            print(f"ID: {snippet['id']}")
            print(f"code: {snippet['code']}")
            print(f"Cosine Similarity (0-1): {snippet['similarity']:.4f}")
            print("-" * 50)

        return similar_code_snippets

    except Exception as e:
        print(f"An error occurred: {e}")
        return []


In [60]:
code =  """
 def div_list(nums1,nums2):
  result = map(lambda x, y: x / y, nums1, nums2)
  return list(result)
  """

In [ ]:
# Call the function code t5
similar_code = Get_similar_Code(code, "codet5",model_codet5, tokenizer_codet5,"/content/drive/MyDrive/Chroma DB/python_code/python_codet5_code_Vdb","python_codet5_code_collection")

emdeeeeiing
ID: 618
code: def div_list(nums1,nums2):
  result = map(lambda x, y: x / y, nums1, nums2)
  return list(result)
Cosine Similarity (0-1): 1.0000
--------------------------------------------------
ID: 682
code: def mul_list(nums1,nums2):
  result = map(lambda x, y: x * y, nums1, nums2)
  return list(result)
Cosine Similarity (0-1): 0.9501
--------------------------------------------------
ID: 358
code: def moddiv_list(nums1,nums2):
  result = map(lambda x, y: x % y, nums1, nums2)
  return list(result)
Cosine Similarity (0-1): 0.9498
--------------------------------------------------
ID: 729
code: def add_list(nums1,nums2):
  result = map(lambda x, y: x + y, nums1, nums2)
  return list(result)
Cosine Similarity (0-1): 0.9074
--------------------------------------------------
ID: 282
code: def sub_list(nums1,nums2):
  result = map(lambda x, y: x - y, nums1, nums2)
  return list(result)
Cosine Similarity (0-1): 0.8945
--------------------------------------------------
ID: 308
co

In [14]:
# Call the function code t5
similar_code = Get_similar_Code(code, "qween",model_qween, tokenizer_qween,"/content/drive/MyDrive/Chroma DB/python_code/python-qwen-VectorDB","pythonVectorDataBase-Qwen2.5-Coder-7B")

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm):

In [16]:
text = """
def div_list(nums1,nums2):
  result = map(lambda x, y: x / y, nums1, nums2)
  return list(result)


[Test Units List]


['assert div_list([4,5,6],[1, 2, 3])==[4.0,2.5,2.0]', 'assert div_list([3,2],[1,4])==[3.0, 0.5]', 'assert div_list([90,120],[50,70])==[1.8, 1.7142857142857142]']
"""

In [20]:
print(text.split("[Test Units List]")[1])




['assert div_list([4,5,6],[1, 2, 3])==[4.0,2.5,2.0]', 'assert div_list([3,2],[1,4])==[3.0, 0.5]', 'assert div_list([90,120],[50,70])==[1.8, 1.7142857142857142]']



In [67]:
# Call the function graph code bert
similar_code = Get_similar_Code(code, "graphcodebert",model_graphBert, tokenizer_graphBert,"/content/drive/MyDrive/Chroma DB/python_code/Python_VDB_Graph_Code_BERT","codePY")

graphcodebert
emdeeeeiing
ID: 617
code: def div_list(nums1,nums2):
  result = map(lambda x, y: x / y, nums1, nums2)
  return list(result)
Cosine Similarity (0-1): 1.0000
--------------------------------------------------
ID: 357
code: def moddiv_list(nums1,nums2):
  result = map(lambda x, y: x % y, nums1, nums2)
  return list(result)
Cosine Similarity (0-1): 0.9695
--------------------------------------------------
ID: 681
code: def mul_list(nums1,nums2):
  result = map(lambda x, y: x * y, nums1, nums2)
  return list(result)
Cosine Similarity (0-1): 0.8388
--------------------------------------------------
ID: 281
code: def sub_list(nums1,nums2):
  result = map(lambda x, y: x - y, nums1, nums2)
  return list(result)
Cosine Similarity (0-1): 0.7981
--------------------------------------------------
ID: 728
code: def add_list(nums1,nums2):
  result = map(lambda x, y: x + y, nums1, nums2)
  return list(result)
Cosine Similarity (0-1): 0.7946
-----------------------------------------------

# DESC

Falcon , codet5

In [ ]:
#Code t5 desc (Sameh Notebook)
def get_embeddings_CodeT5_Desc(code,model,tokenizer):
    inputs = tokenizer(code, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        encoder_outputs = model.encoder(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"]
        )
    return encoder_outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [ ]:
tokenizer_falcon = AutoTokenizer.from_pretrained("Rocketknight1/falcon-rw-1b")
model_falcon = AutoModel.from_pretrained("Rocketknight1/falcon-rw-1b")

#Falcon desc (Rawan Notebook)
def get_embeddings_Falcon_Desc(texts,model, tokenizer):

    inputs = tokenizer([texts], return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    embeddings = outputs.hidden_states[-1].float().mean(dim=1).cpu().numpy()
    return embeddings

tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

In [ ]:
def Get_similar_Desc(description,model_name , model, tokenizer, db_path, collection_name):
    try:
        client = chromadb.PersistentClient(path=db_path)
        collection = client.get_collection(name=collection_name)

        if model_name.lower() == "falcon":
            embedding = get_embeddings_Falcon_Desc(description, model,tokenizer)
        elif model_name.lower() == "codet5":
            embedding = get_embeddings_CodeT5_Desc(description, model,tokenizer)
        else:
            raise ValueError(f"Unsupported model name: {model_name}")

        embedding_list = embedding.flatten().tolist()

        total_documents = collection.count()

        results = collection.query(
            query_embeddings=[embedding_list],
            n_results=total_documents
        )

        distances = results["distances"][0]
        min_distance = min(distances)
        max_distance = max(distances)

        def normalize_distance(distance):
            # Normalize distances to the range [0, 1]
            return 1 - ((distance - min_distance) / (max_distance - min_distance))

        # Process the results
        similar_code_snippets = [
            {
                "id": item_id,
                "description": document,
                "similarity": normalize_distance(distance)
            }
            for item_id, document, distance in zip(
                results["ids"][0],
                results["documents"][0],
                results["distances"][0]
            )
        ]

        # Sort by cosine similarity (descending order)
        similar_code_snippets.sort(key=lambda x: x["similarity"], reverse=True)

        for snippet in similar_code_snippets:
            print(f"ID: {snippet['id']}")
            print(f"Description: {snippet['description']}")
            print(f"Cosine Similarity (0-1): {snippet['similarity']:.4f}")
            print("-" * 50)

        return similar_code_snippets

    except Exception as e:
        print(f"An error occurred: {e}")
        return []


In [ ]:
description = "Write a function to find the length of the longest sub-sequence such that elements in the subsequences are consecutive integers."

In [ ]:
# Call the function
similar_snippets = Get_similar_Desc(description, "falcon",model_falcon, tokenizer_falcon,"/content/drive/MyDrive/Chroma DB/Desc_Falcon/Chroma_DB_Py_Desc","Python-VectorDB-Desc-Falcon")

ID: 863
Description: Write a function to find the length of the longest sub-sequence such that elements in the subsequences are consecutive integers.
Cosine Similarity (0-1): 1.0000
--------------------------------------------------
ID: 548
Description: Write a function to find the length of the longest increasing subsequence of the given sequence.
Cosine Similarity (0-1): 0.8978
--------------------------------------------------
ID: 60
Description: Write a function to find the maximum length of the subsequence with difference between adjacent elements for the given array.
Cosine Similarity (0-1): 0.8871
--------------------------------------------------
ID: 149
Description: Write a function to find the longest subsequence such that the difference between adjacents is one for the given array.
Cosine Similarity (0-1): 0.8869
--------------------------------------------------
ID: 923
Description: Write a function to find the length of the shortest string that has both str1 and str2 as su

In [ ]:
# Call the function code t5
similar_code = Get_similar_Desc(description, "codet5",model_codet5, tokenizer_codet5,"/content/drive/MyDrive/Chroma DB/python_codet5_disc_Vdb","python_codet5_disc_collection")

Streaming output truncated to the last 5000 lines.
Cosine Similarity (0-1): 0.7851
--------------------------------------------------
ID: 305
Description: Write a function to match two words from a list of words starting with letter 'p'.
Cosine Similarity (0-1): 0.7847
--------------------------------------------------
ID: 634
Description: Write a python function to find the sum of fourth power of first n even natural numbers.
Cosine Similarity (0-1): 0.7846
--------------------------------------------------
ID: 482
Description: Write a function to find sequences of one upper case letter followed by lower case letters in the given string by using regex.
Cosine Similarity (0-1): 0.7843
--------------------------------------------------
ID: 378
Description: Write a python function to shift last element to first position in the given list.
Cosine Similarity (0-1): 0.7842
--------------------------------------------------
ID: 710
Description: Write a function to access the initial and last

# Delete Indexing

In [4]:
import random
import chromadb
import os

In [5]:
def delete_random_indices(db_path, collection_name, num_to_delete, output_dir="deleted_ids_folder"):

    # Step 1: Load the Chroma DB client using PersistentClient with the correct path
    client = chromadb.PersistentClient(path=db_path)

    # Step 2: Retrieve the collection
    collection = client.get_collection(collection_name)

    # Step 3: Get all document IDs from the collection by fetching all documents
    all_indices = collection.get(ids=None)  # This retrieves all documents, including their IDs
    document_ids = all_indices['ids']  # Extract document IDs

    # Count the number of documents before deletion
    initial_count = len(document_ids)
    print(f"Initial number of documents: {initial_count}")

    if len(document_ids) < num_to_delete:
        print(f"Not enough indices to delete. There are only {len(document_ids)} indices.")
        return

    # Step 4: Choose a random number of indices to delete
    indices_to_delete = random.sample(document_ids, num_to_delete)

    # Step 5: Delete the selected indices
    for index in indices_to_delete:
        collection.delete(ids=[index])

    # Step 6: Save deleted IDs to a file in the specified directory
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists
    output_file_path = os.path.join(output_dir, "deleted_ids.txt")  # Construct the file path

    with open(output_file_path, "a") as file:
        file.write("\n".join(indices_to_delete) + "\n")

    # Step 7: Count the number of documents after deletion
    all_indices_after_deletion = collection.get(ids=None)  # Fetch remaining documents
    remaining_document_ids = all_indices_after_deletion['ids']
    remaining_count = len(remaining_document_ids)

    print(f"Deleted {num_to_delete} random indices.")
    print(f"Remaining number of documents: {remaining_count}")
    print(f"Deleted IDs saved to {output_file_path}.")


In [30]:
def delete_ids_that_from_file(db_path, collection_name, input_file="deleted_ids_folder/deleted_ids.txt"):
    """
    Reads IDs from a file and deletes them from a specified collection.

    Args:
        db_path (str): Path to the Chroma DB database.
        collection_name (str): Name of the collection to modify.
        input_file (str): Path to the file containing IDs to delete.
    """
    # Step 1: Ensure the file exists
    if not os.path.exists(input_file):
        print(f"File {input_file} does not exist.")
        return

    # Step 2: Load the IDs from the file
    with open(input_file, "r") as file:
        ids_to_delete = file.read().splitlines()

    if not ids_to_delete:
        print("No IDs found in the file to delete.")
        return

    print(f"Loaded {len(ids_to_delete)} IDs from {input_file}.")

    # Step 3: Load the Chroma DB client using PersistentClient with the correct path
    client = chromadb.PersistentClient(path=db_path)

    # Step 4: Retrieve the collection
    collection = client.get_collection(collection_name)

    # Step 5: Delete the IDs from the collection
    for doc_id in ids_to_delete:
        collection.delete(ids=[doc_id])

    print(f"Deleted {len(ids_to_delete)} IDs from the collection '{collection_name}'.")


In [46]:
db_path = "/content/drive/MyDrive/Chroma DB/Desc_Falcon/Chroma_DB_Py_Desc"

In [47]:
client_py = chromadb.PersistentClient(path=db_path)

In [48]:
collections = client_py.list_collections()


In [49]:
collections

[Collection(name=Python-VectorDB-Desc-Falcon)]

In [28]:
collection_name = "Python-VectorDB-Desc-Falcon"

In [29]:
removed_count = delete_random_indices(db_path, collection_name,100)

Initial number of documents: 1138
Deleted 100 random indices.
Remaining number of documents: 1038
Deleted IDs saved to deleted_ids_folder/deleted_ids.txt.


In [39]:
delete_ids_that_from_file(
    db_path="/content/drive/MyDrive/Chroma DB/Desc_Falcon/python_codet5_code_Vdb",
    collection_name="python_codet5_code_collection"
)

Loaded 100 IDs from deleted_ids_folder/deleted_ids.txt.
Deleted 100 IDs from the collection 'python_codet5_code_collection'.


In [50]:
client = chromadb.PersistentClient(path="/content/drive/MyDrive/Chroma DB/Desc_Falcon/python_codet5_code_Vdb")
collection = client.get_collection("python_codet5_code_collection")

In [53]:
remaining_docs = collection.get()

In [54]:
print("Remaining documents:", remaining_docs)

Remaining documents: {'ids': ['1', '2', '3', '4', '5', '6', '8', '9', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '52', '54', '55', '56', '57', '58', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '72', '74', '75', '76', '77', '78', '79', '80', '83', '84', '85', '86', '87', '88', '90', '91', '92', '94', '95', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '127', '128', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '